In [1]:
from sqlite_transactions import *

2018-07-15 13:15:12,354 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-15 13:15:12,355 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 13:15:12,356 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-15 13:15:12,356 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 13:15:12,357 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-15 13:15:12,358 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 13:15:12,360 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE my_table (
	"PK" INTEGER NOT NULL, 
	"X" INTEGER, 
	PRIMARY KEY ("PK")
)


2018-07-15 13:15:12,360 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 13:15:12,361 INFO sqlalchemy.engine.base.Engine COMMIT


In [2]:
engine.echo = False  # examples illustrate DB operations

Database is empty.

In [3]:
select_all()

[]

Expected behavior of "atomic" functions.

In [4]:
i_1()
select_all()

[(1, 1)]

In [5]:
f_1()
select_all()

Exception: 

In [6]:
i_2()
select_all()

[(1, 1), (2, 2)]

In [7]:
f_2()
select_all()

Exception: 

# Try out the different composition functions

In [8]:
select_all()  # the database already has two entries

[(1, 1), (2, 2)]

## `many_all`

All transactions are commited if and only all nested transaction are successful and if there are no other errors.

In [9]:
many_all([i_1, f_2])
select_all()

[(1, 1), (2, 2)]

In [10]:
many_all([f_1, i_2])
select_all()

[(1, 1), (2, 2)]

In [11]:
many_all([i_1, i_2, Exception])
select_all()

[(1, 1), (2, 2)]

In [12]:
many_all([f_1, Exception, i_2])
select_all()

[(1, 1), (2, 2)]

In [13]:
many_all([i_1, i_2])
select_all()

[(1, 1), (2, 2), (3, 1), (4, 2)]

## many_any

Each successful transaction is committed independently of the state of other transactions.

In [14]:
many_any([i_1, f_2])
select_all()

[(1, 1), (2, 2), (3, 1), (4, 2), (5, 1)]

In [15]:
many_any([f_1, i_2])
select_all()

[(1, 1), (2, 2), (3, 1), (4, 2), (5, 1), (6, 2)]

In [16]:
many_any([i_1, i_2, Exception])
select_all()

[(1, 1), (2, 2), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 2)]

In [17]:
many_any([f_1, Exception, i_2])
select_all()

[(1, 1), (2, 2), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 2), (9, 2)]

In [18]:
many_any([i_1, i_2])
select_all()

[(1, 1),
 (2, 2),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 2),
 (9, 2),
 (10, 1),
 (11, 2)]

## many_checkpoint

Succesful transactions are commited until the first failed transaction, which is rolled back.

In [19]:
many_checkpoint([i_1, f_2])
select_all()

[(1, 1),
 (2, 2),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 2),
 (9, 2),
 (10, 1),
 (11, 2),
 (12, 1)]

In [20]:
many_checkpoint([f_1, i_2])
select_all()

[(1, 1),
 (2, 2),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 2),
 (9, 2),
 (10, 1),
 (11, 2),
 (12, 1)]

In [21]:
many_checkpoint([i_1, Exception, i_2])
select_all()

[(1, 1),
 (2, 2),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 2),
 (9, 2),
 (10, 1),
 (11, 2),
 (12, 1),
 (13, 1)]

In [22]:
many_checkpoint([i_1, i_2])
select_all()

[(1, 1),
 (2, 2),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 2),
 (9, 2),
 (10, 1),
 (11, 2),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 2)]